In [1]:
import numpy as np
import tensorflow as tf
import os
import sys
from gensim.models import Word2Vec
import math

**Define functions**

In [2]:
# Create placeholders
X = tf.placeholder(shape=(None, None), dtype=tf.int32)
Y = tf.placeholder(shape=(None), dtype=tf.int32)

In [3]:
# Create RNN
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
        :param batch_size: Size of batches
        :param rnn_size: Size of RNNs
        :return: Tuple (cell, initialize state)
    """
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
    initial_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    return lstm_cell, initial_state

def build_rnn(cell, inputs, num_classes):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    logits = tf.contrib.layers.fully_connected(outputs, num_classes, 
                                               activation_fn=None,
                                               weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.zeros_initializer)
    return logits, final_state

In [4]:
# Create loss function
def loss_function(labels, logits):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    return tf.reduce_mean(loss)

**Get data**

In [5]:
import pickle
with open('../data/embedding_matrix.npy', 'r') as f:
    embedding_matrix = pickle.load(f)
embedding_matrix.shape

(82935, 100)

In [6]:
# Create embedding lookup
embeddings = tf.constant(embedding_matrix, dtype=tf.float32)
X_embedded = tf.nn.embedding_lookup(embeddings, X)
X_embedded.shape

TensorShape([Dimension(None), Dimension(None), Dimension(100)])

In [7]:
with open('../data/batch_data.npy', 'r') as f:
    batch_data = pickle.load(f)

with open('../data/batch_label.npy', 'r') as f:
    batch_label = pickle.load(f)

**Build the graph**

In [8]:
num_classes = 21
num_epochs = 50
batch_size = 32
learning_rate = 0.001
rnn_size = 100

In [9]:
lstm_cell, initial_state = get_init_cell(tf.shape(X)[0], rnn_size)
logits, final_state = build_rnn(lstm_cell, X_embedded, num_classes)
logits = logits[:,-1,:]
probs = tf.nn.softmax(logits)
cost = loss_function(Y, logits)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

**Train**

In [10]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
loss_history = []
for epoch in range(num_epochs):
    print("Epoch {}".format(epoch))
    state = sess.run(initial_state, {X: batch_data[10][:64]})
    for key in batch_data:
        total_data = len(batch_data[key])
        num_batches = int(math.ceil(total_data/float(batch_size)))
        for batch in range(num_batches):
            train_x = batch_data[key][batch*batch_size:(batch+1)*batch_size]
            train_y = batch_label[key][batch*batch_size:(batch+1)*batch_size]
            train_y = train_y.reshape(train_y.shape[0])
            feed = {X: train_x,
                    Y: train_y,
                    initial_state: state}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
    print("Training Loss: {}".format(train_loss))
    loss_history.append(train_loss)

Epoch 0
Training Loss: 0.61088180542
Epoch 1
Training Loss: 0.0522088930011
Epoch 2
Training Loss: 0.0172134563327
Epoch 3
Training Loss: 0.0135728260502
Epoch 4
